In [1]:
import os
import h5py
import numpy as np

save_path = "/media/temp/DATA21/Project_FAST/CODE/MIL_CLIP_Adapter/output_CAMELYON_feat_224x224_5x_CLIP(RN50)"
for split in ['train']:
    all_patches = np.array(h5py.File(os.path.join(save_path, f"{split}_patch_feat.h5"), 'r')['dataset_1'])
    patch_corresponding_slide_label = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_label.npy"))
    patch_corresponding_slide_index = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_index.npy"))
    patch_corresponding_slide_name = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_name.npy"))
    patch_label = np.load(os.path.join(save_path, f"{split}_patch_label.npy"))

    print(split)
    print(f'{split}_all_patches.shape:', all_patches.shape)
    print('\tNaN:', np.isnan(all_patches).sum())
    print(f'{split}_patch_corresponding_slide_label.shape:', patch_corresponding_slide_label.shape)
    print(f'{split}_patch_corresponding_slide_index.shape:', patch_corresponding_slide_index.shape)
    print(f'{split}_patch_corresponding_slide_name.shape:', patch_corresponding_slide_name.shape)
    print(f'{split}_patch_label.shape:', patch_label.shape)
    print()

    num_patches = all_patches.shape[0]
    num_slides = patch_corresponding_slide_index.max() + 1

    # 2. sort instances features into bag
    slide_index_all = []
    slide_feat_all = []
    slide_label_all = []
    slide_patch_label_all = []
    for i in range(num_slides):
        slide_index_all.append(i)
        idx_from_same_slide = patch_corresponding_slide_index == i
        idx_from_same_slide = np.nonzero(idx_from_same_slide)[0]

        slide_feat_all.append(all_patches[idx_from_same_slide])
        assert (patch_corresponding_slide_label[idx_from_same_slide].max() == 
                patch_corresponding_slide_label[idx_from_same_slide].min())
        slide_label_all.append(patch_corresponding_slide_label[idx_from_same_slide].max())
        slide_patch_label_all.append(patch_label[idx_from_same_slide])
    print(f"[DATA INFO] num_slide is {num_slides}; num_patches is {num_patches}")
    print(f"[DATA INFO] #pos_slide is {sum(slide_label_all)}, #neg_slide is {len(slide_label_all) - sum(slide_label_all)}")

train
train_all_patches.shape: (568842, 1024)
	NaN: 0
train_patch_corresponding_slide_label.shape: (568842,)
train_patch_corresponding_slide_index.shape: (568842,)
train_patch_corresponding_slide_name.shape: (568842,)
train_patch_label.shape: (568842,)

[DATA INFO] num_slide is 251; num_patches is 568842
[DATA INFO] #pos_slide is 92, #neg_slide is 159


In [2]:
print(len(slide_index_all), len(slide_feat_all), len(slide_label_all), len(slide_patch_label_all))
for index, f, l, pl in zip(slide_index_all, slide_feat_all, slide_label_all, slide_patch_label_all):
    print(f"Slide Index: {index:03d}\t Slide Feature: {f.shape}\t Slide Label: {l}\t Slide Patch Label: {pl.shape}\t")

251 251 251 251
Slide Index: 000	 Slide Feature: (4155, 1024)	 Slide Label: 0	 Slide Patch Label: (4155,)	
Slide Index: 001	 Slide Feature: (756, 1024)	 Slide Label: 0	 Slide Patch Label: (756,)	
Slide Index: 002	 Slide Feature: (2179, 1024)	 Slide Label: 0	 Slide Patch Label: (2179,)	
Slide Index: 003	 Slide Feature: (1971, 1024)	 Slide Label: 1	 Slide Patch Label: (1971,)	
Slide Index: 004	 Slide Feature: (1583, 1024)	 Slide Label: 0	 Slide Patch Label: (1583,)	
Slide Index: 005	 Slide Feature: (5383, 1024)	 Slide Label: 1	 Slide Patch Label: (5383,)	
Slide Index: 006	 Slide Feature: (600, 1024)	 Slide Label: 0	 Slide Patch Label: (600,)	
Slide Index: 007	 Slide Feature: (2637, 1024)	 Slide Label: 0	 Slide Patch Label: (2637,)	
Slide Index: 008	 Slide Feature: (1923, 1024)	 Slide Label: 0	 Slide Patch Label: (1923,)	
Slide Index: 009	 Slide Feature: (1821, 1024)	 Slide Label: 0	 Slide Patch Label: (1821,)	
Slide Index: 010	 Slide Feature: (1623, 1024)	 Slide Label: 0	 Slide Patch Lab

In [ ]:
import sys
sys.path.append('../')

import torch
from models.learnable_prompt import Instance_CLIP, PromptLearner
from prompt_learning.prompts import get_patch_level_prompts_forCAMELYON
from prompt_learning.util import (simple_instance_dataset, Optimizer_TextBranch, 
                                  gather_instance_prediction_and_pred_bag, 
                                  norm_logit, cal_matrix_mul, build_MIL_Adapter_cache_model, 
                                  load_clip_to_cpu, clip_classifier)

clip_model = load_clip_to_cpu(backbone_name='RN50').cuda()
(prompts_common_templates, 
prompts_pathology_template, 
prompts_pathology_template_withDescription) = get_patch_level_prompts_forCAMELYON(tissue_type='simple')
classifer_common_templates = clip_classifier(prompts_common_templates, clip_model)
classifer_pathology_template = clip_classifier(prompts_pathology_template, clip_model)
classifer_pathology_template_withDescription = clip_classifier(prompts_pathology_template_withDescription, clip_model)

learnablePrompt_ctx_init = [
    # "A normal image patch with regularly shaped cells and smaller, lighter nuclei. * * * * * * * * * *",
    # "A tumor image patch with irregular cancerous cells and larger, darker nuclei. * * * * * * * * * *",
    "An image of evenly spaced, regular cells, suggesting healthy tissue. * * * * * * * * * *",
    # "Tissue with orderly and uniform cells, indicating no cancer.",
    "An image of disorganized, dense, and irregular cells, suggesting cancer. * * * * * * * * * *"
    # "Tissue with abnormal cells and chaotic structure, indicating malignancy.",
]
instance_prompt_learner = PromptLearner(n_ctx=16, ctx_init=learnablePrompt_ctx_init, all_ctx_trainable=True, csc=True,
                                        classnames=["normal", "tumor"],
                                        clip_model='RN50', p_drop_out=0.0)
model_text = Instance_CLIP(instance_prompt_learner).to('cuda:0')
for param in model_text.text_encoder.parameters():
    param.requires_grad = False
optimizer_text_branch = torch.optim.SGD(model_text.prompt_learner.parameters(), lr=0.001)

train_ds_fewShotInstance = simple_instance_dataset(feat=cache_keys_labeled, label=cache_values_labeled)
val_ds_allInstance = simple_instance_dataset(feat=test_features, label=test_labels)
train_loader_fewShotInstance = torch.utils.data.DataLoader(train_ds_fewShotInstance, batch_size=1024, shuffle=True, num_workers=0, drop_last=False)
val_loader_allInstance = torch.utils.data.DataLoader(val_ds_allInstance, batch_size=1024, shuffle=False, num_workers=0, drop_last=False)

optimizer = Optimizer_TextBranch(
    model=model_text, train_loader=train_loader_fewShotInstance, test_loader=val_loader_allInstance,
    optimizer=optimizer_text_branch, writer=writer, num_epoch=50,
    dev=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer.optimize()
classifer_pathology_after_optimizing = model_text.get_classifier().detach()

/media/temp/DATA21/Project_FAST/CODE/FALST/dev_notebook/../clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


In [5]:
classifer_pathology_template_withDescription.shape

torch.Size([2, 1024])

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

instance_shots = [4, 16, 64]

result = pd.DataFrame(columns=[4, 16, 64])

with torch.no_grad():
    for idx, f, l, pl in zip(slide_index_all, slide_feat_all, slide_label_all, slide_patch_label_all):
        if l == 0:
            continue
        slide_feat = torch.tensor(f).cuda()
        patch_label = torch.tensor(pl).cuda()
        text_feat = classifer_pathology_template_withDescription.cuda()
        clip_logits = slide_feat @ text_feat.T
        # clip_logits -= clip_logits.min()
        # clip_logits /= clip_logits.max()
        clip_probs = F.softmax(clip_logits, dim=-1)
        for instance_shot in [4, 16, 64]:
            top_instance_idx = clip_probs[:, 1].topk(instance_shot, dim=0)[1]
            top_instance_label = patch_label[top_instance_idx]
            accuracy = top_instance_label.sum().float() / instance_shot
            result.loc[idx, instance_shot] = accuracy.item()

In [11]:
clip_probs

tensor([[0.4979, 0.5021],
        [0.4989, 0.5011],
        [0.4987, 0.5013],
        ...,
        [0.4974, 0.5026],
        [0.4988, 0.5012],
        [0.4999, 0.5001]], device='cuda:0')

In [9]:
pd.set_option('display.max_rows', None)  # 显示所有行
result

,4,16,64
3,0.0,0.0,0.03125
5,0.25,0.375,0.25
11,0.0,0.0,0.0
12,0.0,0.125,0.09375
18,0.0,0.0,0.0
21,0.0,0.125,0.21875
23,0.0,0.0,0.0
25,0.25,0.75,0.796875
26,0.0,0.0,0.015625
27,0.0,0.0,0.046875


In [67]:
top_instance_idx

tensor([2024,   64,  227,  957,  275, 1135,  495,  518, 1334,  113,  634, 1140,
         546, 1305,  508, 1495, 1684, 1847, 1164, 1540, 1900, 1318, 1098,  665,
         374, 1032, 1125, 1199,  189, 1547, 1163, 1134,  194, 1997, 1700, 1418,
         590,  293,  763,  432, 1246,  368, 1103, 1933, 1147,  927,  668, 1356,
         484,   90,  713, 1104, 1439,  404,    9,   39, 1783,  333,  125,  652,
        1425,  797, 1602, 1676], device='cuda:0')

In [58]:
torch.topk(clip_probs[:, 1], 5, dim=0)

torch.return_types.topk(
values=tensor([0.5496, 0.5479, 0.5459, 0.5442, 0.5436], device='cuda:0'),
indices=tensor([3892, 3064, 1562, 2375, 1220], device='cuda:0'))

In [55]:
clip_probs

tensor([[0.5027, 0.4973],
        [0.4993, 0.5007],
        [0.4976, 0.5024],
        ...,
        [0.4989, 0.5011],
        [0.4980, 0.5020],
        [0.5010, 0.4990]], device='cuda:0')